# Risk Parity Portfolio Optimization Application

Constructing a portfolio of stocks using Risk Parity methods for optimal performance of backtested risk measures.  

Features of the application:

* Inputs stock tickers and share/dollar value of existing portfolio for baseline measurement

* Runs backtests against 9 risk measures to measure performance against existing portolio  

* Calculates reallocation of existing portfolio to a selected risk measure 

In [24]:
import riskfolio as rp 
import pandas as pd
import numpy as np
from openbb_terminal.sdk import openbb
import datetime
from dateutil.relativedelta import relativedelta
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import yfinance as yf
import bt 
import matplotlib.pyplot as plt
%matplotlib inline



# Read inputed portfolio from Google Sheets using Cloud API

index.htlm tempate allows users to input stock tickers and a dollar/share value of each stock. From there we calculate the allocation % of the current portfolio. 

In [26]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(2)

data = sheet_instance.get_all_values()

portfolio = pd.DataFrame(data)

headers = portfolio.iloc[0]
portfolio = pd.DataFrame(portfolio.values[1:], columns=headers)

portfolio = portfolio.rename(columns= {'':'Ticker'})

portfolio['DollarValue'] = portfolio['Shares'].astype(float)

total_shares = portfolio['DollarValue'].sum()

portfolio['% Allocation'] = (portfolio['DollarValue'] / total_shares)

portfolio



,Ticker,Shares,DollarValue,% Allocation
0,CEG,2108,2108.0,0.108576
1,AAPL,1727,1727.0,0.088952
2,ULTA,1685,1685.0,0.086789
3,MSFT,1657,1657.0,0.085346
4,RTX,1623,1623.0,0.083595
5,JPM,1321,1321.0,0.068040
6,CVX,1276,1276.0,0.065722
7,PANW,1225,1225.0,0.063096
8,GEHC,1141,1141.0,0.058769
9,AMZN,1080,1080.0,0.055627


# Initiate stock market data from OpenBB

Talking last 1 Year stock data of portfolio tickers

In [3]:
end = pd.Timestamp(datetime.date.today())
start = end - relativedelta(years=1)

symbols = portfolio['Ticker'].tolist()

tickers = openbb.stocks.ca.hist(symbols,start, end)

returns = tickers.pct_change()[1:]
returns.dropna(how="any", axis=1, inplace=True)

NaN values found in: NVDA.  Backfilling data

# Riskfolio Lib for Risk Parity evaluation of 9 risk measures

Building a function to run portfolio stock tickers thru different risk measures to define allocation for each risk measure

In [27]:
risk_measures = ['MV', 'SLPM', 'CVaR','MAD','FLPM','EVaR','UCI','CDaR','MSV']

weights = pd.DataFrame([])

# Create an instance of the Portfolio class
P = rp.Portfolio(returns=returns,)

# Define constraints
P.assets_stats(method_mu='hist', method_cov='hist', d=0.94)

P.lowerret = 0.0015

for rm in risk_measures:

    w_rp = P.rp_optimization(model='Classic', rm = rm, b=None)
    
    weights = pd.concat([weights, w_rp], axis=1)

weights.columns = risk_measures

weights_deindex = weights.reset_index()

weights_deindex.rename(columns={'index':'Ticker'}, inplace=True )

weights_deindex

,Ticker,MV,SLPM,CVaR,MAD,FLPM,EVaR,UCI,CDaR,MSV
0,CEG,0.086885,0.089325,0.082801,0.089445,0.093569,0.084966,0.202633,0.227039,0.087651
1,AAPL,0.069475,0.071768,0.083512,0.070711,0.069241,0.087175,0.061670,0.059171,0.071233
2,ULTA,0.060821,0.064556,0.064173,0.058918,0.059907,0.079559,0.046051,0.049623,0.063438
3,MSFT,0.086047,0.089662,0.090425,0.085406,0.087961,0.088924,0.054485,0.047906,0.087573
4,RTX,0.062623,0.066676,0.061035,0.059666,0.064496,0.058077,0.043875,0.053271,0.065581
5,JPM,0.082222,0.081839,0.075756,0.077184,0.076675,0.079289,0.066501,0.063741,0.079800
6,CVX,0.055955,0.057147,0.056216,0.061044,0.054892,0.053703,0.102287,0.078567,0.057667
7,PANW,0.084125,0.070199,0.064513,0.081208,0.085033,0.055234,0.100288,0.101726,0.072799
8,GEHC,0.052245,0.055564,0.062136,0.049394,0.051896,0.042353,0.065729,0.062885,0.055312
9,AMZN,0.065892,0.056202,0.055947,0.059711,0.054661,0.061830,0.044204,0.039894,0.057802


# Write risk measure allocations to Google Sheet database 

In [5]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(1)

#mapping_list =weights_deindex.values.tolist()

#sheet.values_append('RM Weights!A1',{'valueInputOption' : 'RAW'}, {'values':mapping_list})

set_with_dataframe(sheet_instance,weights_deindex)

# Load stock market data using YFinance 

Initiating portfolio values and S&P500 data for risk measure comparison 

In [6]:
stock_data = yf.download(symbols, start= start, end= end)['Adj Close']

stock_data.dropna(how="any", axis=1, inplace=True)

port_allocation = portfolio['% Allocation'].tolist()

port_list = portfolio['% Allocation'].tolist()

port_values = portfolio['% Allocation'].values

# Fetch the S&P 500 index data
sp500_data = yf.download('^GSPC', start= start, end= end)['Adj Close']

sp500_data = sp500_data.to_frame(name='S&P500')


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed


# Build function to backtest each risk measure, current portfolio and S&P500

In [7]:
results = {}

# Assuming stock_data is a DataFrame with historical price data for each asset
# stock_data.columns should match the assets in the weights DataFrame

for rm in risk_measures:
    # Extracting the allocation for the given risk measure
    allocation = weights[rm].values

    # Ensuring the weights sum up to 1 (if they don't due to rounding or other reasons)
    allocation = allocation/allocation.sum()

    # Create the portfolio strategy for the given risk measure
    portfolio_weights = dict(zip(stock_data, allocation))
    
    portfolio_strategy = bt.Strategy( f'Portfolio_{rm}',
                                     algos=[
                                         bt.algos.SelectAll(),
                                         bt.algos.WeighSpecified(**portfolio_weights),
                                         bt.algos.Rebalance()
                                     ])

    # Backtest the portfolio strategy
    portfolio_backtest = bt.Backtest(portfolio_strategy, stock_data)
    res = bt.run(portfolio_backtest)

    # Store the backtest result for the risk measure in the results dictionary
    results[rm] = res

 
stats_dict = {}

for rm, res in results.items():
    # Extracting stats for each backtest. This returns a pandas Series
    stats_series = res.stats
    
    # Store the Series in the dictionary with risk measure as the key
    stats_dict[rm] = stats_series

stats_df = pd.concat(stats_dict, axis=1).T

# Transpose again to swap the index with the columns
stats_df = stats_df.T

stats_df.columns = stats_df.columns.get_level_values(0) 

RM_results = stats_df 


   

In [8]:
# Create the portfolio strategy
portfolio_weights = dict(zip(stock_data, port_values))
portfolio_strategy = bt.Strategy('MyPortfolio', 
                                  algos=[
                                      bt.algos.SelectAll(),
                                      bt.algos.WeighSpecified(**portfolio_weights),
                                      bt.algos.Rebalance()
                                  ])

# Backtest the portfolio strategy
portfolio_backtest = bt.Backtest(portfolio_strategy, stock_data)


In [9]:
# Create and backtest the S&P 500 benchmark
benchmark_strategy = bt.Strategy('S&P500', [bt.algos.RunOnce(),
                                            bt.algos.SelectAll(),
                                            bt.algos.WeighEqually(),
                                            bt.algos.Rebalance()])
benchmark_backtest = bt.Backtest(benchmark_strategy, sp500_data)

In [10]:
# Create and backtest the Equal Allocation benchmark
equal_benchmark_strategy = bt.Strategy('EqualAllocation', [bt.algos.RunOnce(),
                                            bt.algos.SelectAll(),
                                            bt.algos.WeighEqually(),
                                            bt.algos.Rebalance()])
equal_benchmark_backtest = bt.Backtest(equal_benchmark_strategy, stock_data)

In [11]:
# Run the backtests and compare the results
res = bt.run(portfolio_backtest, equal_benchmark_backtest, benchmark_backtest)

# Example: Get the equity curve for the first backtest strategy
equity = res[0].prices

# Calculate the daily percentage returns
daily_returns = equity.pct_change() * 100

stats = res.stats

df_stats = pd.DataFrame(stats)

BM_results = df_stats


# Refine backtested performance metrics

Running backtests for each risk measure, current portfolio, equally allocated portfolio and S&P500 to evaluate performance

In [12]:
portfolio_df = pd.concat([RM_results, BM_results], axis = 1)

portfolio_df = portfolio_df.iloc[:-33]

backtested_portfolio = portfolio_df.iloc[3:]

backtested_portfolio = backtested_portfolio.reset_index()

backtested_portfolio.rename(columns={'index':'Results'}, inplace=True )

backtested_portfolio



,Results,MV,SLPM,CVaR,MAD,FLPM,EVaR,UCI,CDaR,MSV,MyPortfolio,EqualAllocation,S&P500
0,total_return,0.344936,0.358253,0.378818,0.34717,0.347638,0.399845,0.370121,0.34868,0.352211,0.35499,0.416942,0.205098
1,cagr,0.345209,0.358538,0.379121,0.347445,0.347913,0.400167,0.370416,0.348956,0.35249,0.355272,0.417281,0.205252
2,max_drawdown,-0.115279,-0.112453,-0.10778,-0.117933,-0.114856,-0.101725,-0.122271,-0.123913,-0.113239,-0.105207,-0.119324,-0.102663
3,calmar,2.994542,3.188332,3.517538,2.946124,3.029129,3.933804,3.029473,2.816144,3.112809,3.376898,3.49704,1.999275
4,mtd,0.042587,0.044135,0.046039,0.043564,0.043356,0.048837,0.037025,0.03295,0.042694,0.042204,0.068996,0.026041
5,three_month,0.195676,0.196795,0.202785,0.199987,0.198995,0.200037,0.198218,0.189778,0.195572,0.194539,0.239281,0.168773
6,six_month,0.080949,0.08465,0.094835,0.082217,0.084081,0.1008,0.075606,0.065519,0.082737,0.089734,0.121831,0.071456
7,ytd,0.042587,0.044135,0.046039,0.043564,0.043356,0.048837,0.037025,0.03295,0.042694,0.042204,0.068996,0.026041
8,one_year,0.344936,0.358253,0.378818,0.34717,0.347638,0.399845,0.370121,0.34868,0.352211,0.35499,0.416942,0.205098
9,three_year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Write back performance metrics to Google Sheets database

In [13]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(3)

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,backtested_portfolio)

# Calculate reallocation of existing portfolio to chosen risk measure allocation

CVaR was the best performing risk measure based on backtested resuls. 

The resulting dataframe compares the current portfolio values with CVaR portfolio values and calculates the rebalance. 

The '$ Diff' column represents the buy/sell trades needed to allocate portfolio for CVar. 

The 'Weekly_Contribution' column represents the CVaR dollar value to purchase for each stock based on a $150 weekly contribution. 

In [18]:

merged_df = portfolio.merge(weights_deindex[['Ticker', 'CVaR']], on='Ticker', how='left')

#sum of total portfolio value
sum = merged_df['Shares'].sum()

weekly_invest = 150

merged_df['CVaR Allocation'] = merged_df['CVaR'] * sum

merged_df['$ Diff'] = merged_df['CVaR Allocation'] - merged_df['Shares']

merged_df['Weekly_Contribution'] = merged_df['CVaR'] * weekly_invest

merged_df['date'] = pd.Timestamp.now().date()

merged_df = merged_df.sort_values(by='$ Diff', ascending = True)

merged_df



,Ticker,Shares,% Allocation,CVaR,CVaR Allocation,$ Diff,Weekly_Contribution,date
0,CEG,2108.0,0.108576,0.070921,1376.925306,-731.074694,10.638104,2024-01-26
7,PANW,1225.0,0.063096,0.039061,758.375903,-466.624097,5.859201,2024-01-26
1,AAPL,1727.0,0.088952,0.080043,1554.042536,-172.957464,12.006509,2024-01-26
13,NOW,857.0,0.044141,0.036760,713.687062,-143.312938,5.513936,2024-01-26
12,NVDA,1004.0,0.051713,0.046217,897.310174,-106.689826,6.932605,2024-01-26
3,MSFT,1657.0,0.085346,0.081443,1581.213576,-75.786424,12.216432,2024-01-26
9,AMZN,1080.0,0.055627,0.053640,1041.427960,-38.572040,8.046057,2024-01-26
4,RTX,1623.0,0.083595,0.084394,1638.514241,15.514241,12.659137,2024-01-26
5,JPM,1321.0,0.068040,0.072817,1413.750282,92.750282,10.922614,2024-01-26
2,ULTA,1685.0,0.086789,0.095052,1845.433037,160.433037,14.257788,2024-01-26


# Write back reallocation results to Google Sheets database

This record will serve as a trade log for the portfolio. 

By logging CVaR portfolio transactions on this date we can run updated Risk Parity risk measures quarterly to ensure portfolio is optimially balanced. 

In [20]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(0)

existing_data = sheet_instance.get_all_records()
start_row = len(existing_data) + 2

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,merged_df,row=start_row,include_column_header=False)



#mapping_list =merged_df.values.tolist()

#sheet.values_append('Allocation Records!A1',{'valueInputOption' : 'RAW'}, {'values':mapping_list})



